# LSTM decoding exploration

Going to be looking at using LSTMs (with potentially some changes to the cost function) to decode EMGs from cortical data. This is all based off work from Steph and Josh.


Going to start with the old Jango data, then update from there as necessary.

In [41]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.optimize import least_squares
from matplotlib import pyplot as plt
# import ipympl

# we'll use ridge regression as a comparisson
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tkinter import Tk
from tkinter import filedialog as fd # just so I don't have to repeatedly manually enter filenames

# and tf stuff
import tensorflow as tf


In [2]:
%matplotlib qt

### Import data

In [3]:
# request the filename
root = Tk()
mat_fn = fd.askopenfilename(master=root,filetypes=[('matlab data','*.mat')])
root.destroy()

Different loading code if it's data from Josh vs XDS stuff

**Josh**
It looks like this will input a dictionary containing (among other things) numpy arrays for each of the datasets for the day. We'll pull in each of them, and parse accordingly.

In the "binned" numpy arrays (two levels down), the organization seems to be:

0. timestamps
1. metadata
2. EMG names
3. EMG data
4. Force names
5. Force data
6. electrode and unit number
7. channel and unit number
8. Firing Rates (in hz)
9. Kin (cursor kin?) names
10. Kin (cursor kin?) data
11. Vel Data
12. Vel Names
13. Accel Data
14. Accel Names
15. Digital Words and timestamps
16. Targets:

    0. corner locations and appearance time 
    1. rotations and appearance time

17. Trial table data
18. Trial Table Labels
19. 
20. 
21. 

In [4]:
# load it in
data = loadmat(mat_fn)


In [5]:
def load_josh_mat(curr_data):
    timestamps = curr_data[0][0][0].reshape(-1)
    # create a couple of pandas data frames -- this will make things a bit easier

    # EMG
    emg_names = [curr_data[0][0][2][ii][0][0] for ii in np.arange(4)]
    EMG = pd.DataFrame(curr_data[0][0][3], columns=emg_names, index=timestamps)

    # Forces
    force_names = [curr_data[0][0][4][ii][0][0] for ii in np.arange(2)]
    force = pd.DataFrame(curr_data[0][0][5], columns=force_names, index=timestamps)

    # Firing Rates
    channel_names = [f"cc{row[0]:03d}ee{row[1]}" for row in curr_data[0][0][7]]
    firing = pd.DataFrame(curr_data[0][0][8], columns=channel_names, index=timestamps)

    # Kin -- not sure if cursor or ang of wrist. Will need to check
    kin_names = curr_data[0][0][9]
    vel_names = curr_data[0][0][12]
    acc_names = curr_data[0][0][14]
    kin = pd.DataFrame(curr_data[0][0][10], columns=kin_names, index=timestamps)
    kin = kin.join(pd.DataFrame(curr_data[0][0][11], columns=vel_names, index=timestamps))
    kin = kin.join(pd.DataFrame(curr_data[0][0][13], columns=acc_names, index=timestamps))

    # trial table
    trial_names = curr_data[0][0][18]
    trial_table = pd.DataFrame(curr_data[0][0][17], columns=trial_names)

    return timestamps, firing, EMG, force, kin


In [6]:
# load specific data
train_timestamps, train_firing, train_EMG, train_force, train_kin = load_josh_mat(data['WmTrain'])
test_timestamps, test_firing, test_EMG, test_force, test_kin = load_josh_mat(data['WmTest'])




XDS:

## Model building 
Look through a couple of different model types, look to see how they are trained

### linear models
Going to just start with a basic Wiener filter with 10 lags.

In [7]:
# create the lagged input for the wiener filter
wiener_input = pd.DataFrame() # empty dataframe
n_lags = 10 # number of lags
for ii in np.arange(n_lags): # create the lagged dataframe
    col_dict = dict(zip(train_firing.columns, train_firing.columns+f"_lag{ii}"))
    wiener_input = wiener_input.join(train_firing.shift(-ii, fill_value=0).rename(columns=col_dict), how='outer')

wiener_test = pd.DataFrame() # empty dataframe
n_lags = 10 # number of lags
for ii in np.arange(n_lags): # create the lagged dataframe
    col_dict = dict(zip(test_firing.columns, test_firing.columns+f"_lag{ii}"))
    wiener_test = wiener_test.join(test_firing.shift(-ii, fill_value=0).rename(columns=col_dict), how='outer')

# build the model
# mdl = linear_model.LinearRegression() # try with fitting the intersect. I think that's right...
mdl = linear_model.Ridge() # try with ridge regression. Curious which works better
mdl.fit(wiener_input, train_EMG)

# predicted values
emg_preds_train = mdl.predict(wiener_input)
emg_preds_test = mdl.predict(wiener_test)

In [8]:

# plotting the data
fig_lin, ax_lin = plt.subplots(nrows=2, ncols=2, sharex=True, constrained_layout=True)

ax_lin[0,0].plot(train_timestamps,emg_preds_train[:,0], label='Predicted')
ax_lin[0,0].plot(train_timestamps, train_EMG.iloc[:,0], label='Recorded')
ax_lin[0,0].set_title(f"{train_EMG.columns[0]}   vaf: {metrics.r2_score(train_EMG.iloc[:,0], emg_preds_train[:,0])}")
_ = ax_lin[0,0].legend()

ax_lin[1,0].plot(train_timestamps,emg_preds_train[:,1], label='Predicted')
ax_lin[1,0].plot(train_timestamps, train_EMG.iloc[:,1], label='Recorded')
ax_lin[1,0].set_title(f"{train_EMG.columns[1]}   vaf: {metrics.r2_score(train_EMG.iloc[:,1], emg_preds_train[:,1])}")
_ = ax_lin[1,0].legend()

ax_lin[0,1].plot(train_timestamps,emg_preds_train[:,2], label='Predicted')
ax_lin[0,1].plot(train_timestamps, train_EMG.iloc[:,2], label='Recorded')
ax_lin[0,1].set_title(f"{train_EMG.columns[2]}   vaf: {metrics.r2_score(train_EMG.iloc[:,2], emg_preds_train[:,2])}")
_ = ax_lin[0,1].legend()

ax_lin[1,1].plot(train_timestamps,emg_preds_train[:,3], label='Predicted')
ax_lin[1,1].plot(train_timestamps, train_EMG.iloc[:,3], label='Recorded')
ax_lin[1,1].set_title(f"{train_EMG.columns[3]}   vaf: {metrics.r2_score(train_EMG.iloc[:,3], emg_preds_train[:,3])}")
_ = ax_lin[1,1].legend()

In [9]:
# plot test data
fig_lin_test, ax_lin_test = plt.subplots(nrows=2, ncols=2, sharex=True, constrained_layout=True)

ax_lin_test[0,0].plot(test_timestamps,emg_preds_test[:,0], label='Predicted')
ax_lin_test[0,0].plot(test_timestamps, test_EMG.iloc[:,0], label='Recorded')
ax_lin_test[0,0].set_title(f"{test_EMG.columns[0]}   vaf: {metrics.r2_score(test_EMG.iloc[:,0], emg_preds_test[:,0])}")
_ = ax_lin_test[0,0].legend()

ax_lin_test[1,0].plot(test_timestamps,emg_preds_test[:,1], label='Predicted')
ax_lin_test[1,0].plot(test_timestamps, test_EMG.iloc[:,1], label='Recorded')
ax_lin_test[1,0].set_title(f"{test_EMG.columns[1]}   vaf: {metrics.r2_score(test_EMG.iloc[:,1], emg_preds_test[:,1])}")
_ = ax_lin_test[1,0].legend()

ax_lin_test[0,1].plot(test_timestamps,emg_preds_test[:,2], label='Predicted')
ax_lin_test[0,1].plot(test_timestamps, test_EMG.iloc[:,2], label='Recorded')
ax_lin_test[0,1].set_title(f"{test_EMG.columns[2]}   vaf: {metrics.r2_score(test_EMG.iloc[:,2], emg_preds_test[:,2])}")
_ = ax_lin_test[0,1].legend()

ax_lin_test[1,1].plot(test_timestamps,emg_preds_test[:,3], label='Predicted')
ax_lin_test[1,1].plot(test_timestamps, test_EMG.iloc[:,3], label='Recorded')
ax_lin_test[1,1].set_title(f"{test_EMG.columns[3]}   vaf: {metrics.r2_score(test_EMG.iloc[:,3], emg_preds_test[:,3])}")
_ = ax_lin_test[1,1].legend()

### Linear with Static Non-linearity

Using the lab default -- build a wiener filter, fit it, then fit with a static non-linearity on top of the form

$Ax^2 + Bx + C$

Where $x$ is the original EMG prediction, and the output is the new EMG prediction

Also giving the options for a sigmoid or an exponential


**Starting with defining our nonlinearity methods**

In [43]:
# using scipy's least_squares:
def non_linearity(p, y_pred, nonlinear_type='poly'):
    if nonlinear_type == 'poly':
        return p[0] + p[1]*y_pred + p[2]*y_pred**2
    elif nonlinear_type == 'exponential':
        return p[0]*np.exp(p[0]*y_pred)
    elif nonlinear_type == 'sigmoid':
        return 1/(1 + np.exp(-10*(y_pred-p[0])))

def non_linearity_residuals(p, y_pred, y_act, nonlinear_type = 'poly'):
    if nonlinear_type == 'poly':
        return y_act - (p[0] + p[1]*y_pred + p[2]*y_pred**2)
    elif nonlinear_type == 'exponential':
        return y_act - (p[0]*np.exp(p[0]*y_pred))
    elif nonlinear_type == 'sigmoid':
        return y_act - (1/(1 + np.exp(-10*(y_pred-p[0]))))



In [52]:
# n_poly = 4 # degree of the polynomial

nonlinear_type = 'poly'


if nonlinear_type == 'poly':
    init_pred = [.1, .1, .1]
elif nonlinear_type == 'exponential':
    init_pred = [1, .1]
elif nonlinear_type == 'sigmoid':
    init_pred = .5

mdl_A = linear_model.LinearRegression(fit_intercept=True)
mdl_B = {} # a dictionary of numpy polynomials. Probably a better way to do this, maybe Xuan's method... oh well

mdl_A.fit(wiener_input, train_EMG)# fit the first model
prefit_EMG = mdl_A.predict(wiener_input) # get the initially predicted EMGs
prefit_VAF = metrics.r2_score(train_EMG, prefit_EMG, multioutput='raw_values')
## Polynomial
# for ii in np.arange(len(train_EMG.columns)):
#     muscle = train_EMG.columns[ii]
#     mdl_B[muscle] = np.polynomial.Polynomial.fit(prefit_EMG[:,ii]-mdl_A.intercept_[ii], train_EMG.iloc[:,ii]-mdl_A.intercept_[ii], deg=n_poly)
#     print('--------------------------------------------------------')
#     print(muscle)
#     print(f"Linear VAF: {prefit_VAF[ii]}")
#     print(f"NonLinear VAF: {metrics.r2_score(train_EMG.iloc[:,ii],mdl_B[muscle](prefit_EMG[:,ii])+mdl_A.intercept_[ii])}")


for ii in np.arange(len(train_EMG.columns)):
    muscle = train_EMG.columns[ii]
    mdl_B[muscle] = least_squares(non_linearity_residuals, init_pred, args=(prefit_EMG[:,ii], train_EMG.iloc[:,ii].to_numpy(), nonlinear_type))
    print('--------------------------------------------------------')
    print(muscle)
    print(f"Linear VAF: {prefit_VAF[ii]}")
    print(f"NonLinear VAF: {metrics.r2_score(train_EMG.iloc[:,ii],non_linearity(mdl_B[muscle].x,(prefit_EMG[:,ii])))}")

# constants = least_squares(non_linearity_residuals, init_pred, args=(prefit_EMG, train_EMG.to_numpy(), nonlinear_type))



--------------------------------------------------------
FCU
Linear VAF: 0.2428636333628813
NonLinear VAF: 0.28435727716325243
--------------------------------------------------------
FCR
Linear VAF: 0.38902871138196926
NonLinear VAF: 0.45766021381275035
--------------------------------------------------------
ECU
Linear VAF: 0.603803720176696
NonLinear VAF: 0.6154105885380085
--------------------------------------------------------
ECR
Linear VAF: 0.6643602223081486
NonLinear VAF: 0.671607473081607


## Scratch Space
Sometimes the variable viewer isn't very goood

In [11]:
train_EMG.columns

Index(['FCU', 'FCR', 'ECU', 'ECR'], dtype='object')

In [53]:
fig,ax = plt.subplots()

ax.plot(train_timestamps, train_EMG.iloc[:,0], label='Recorded')
ax.plot(train_timestamps, prefit_EMG[:,0], label='Initial Prediction')
ax.plot(train_timestamps, non_linearity(mdl_B['FCR'].x, prefit_EMG[:,0], nonlinear_type='poly'), label='Non-Linear')
ax.plot(train_timestamps, np.maximum(prefit_EMG[:,0],0))

ax.legend()

In [31]:
mdl_B

{'FCU': Polynomial([0.00471293, 0.03770044, 0.03273303, 0.00897191, 0.00289362], domain=[-0.0245153,  0.0378285], window=[-1.,  1.]),
 'FCR': Polynomial([ 0.08630874,  0.38194568,  0.26566659, -0.12920942, -0.12850422], domain=[-0.14492297,  0.3105863 ], window=[-1.,  1.]),
 'ECU': Polynomial([ 0.04329101,  0.21050704,  0.04779372, -0.03414433,  0.00754418], domain=[-0.13909213,  0.23013873], window=[-1.,  1.]),
 'ECR': Polynomial([ 0.03006526,  0.15071669, -0.0271577 , -0.02642181,  0.10646176], domain=[-0.10976289,  0.16909328], window=[-1.,  1.])}

In [30]:
mdl_A.intercept_

0.0

In [37]:
np.maximum(prefit_EMG[:,0],0)

array([0.00851   , 0.01367154, 0.01013622, ..., 0.011456  , 0.01201614,
       0.00841201])

In [51]:
mdl_B['FCU']['x']

array([3.94008562e-03, 8.89894491e-02, 3.33041735e+01])